In [17]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv('../data/real_0_0.csv').sort_values('logB', ascending=False)

In [24]:
def _twin(df, df_query):
    
    def similarity(row):
        # props = ['T', 'data_points']
        # x = np.array(row[props])
        # y = np.array(df_query[props])
        # return np.sqrt(np.sum(np.square(x - y) / np.square(y)))
        d1 = np.abs(row['T'] - df_query['T']) / df_query['T']
        d2 = np.abs(row['data_points'] - df_query['data_points']) / df_query['data_points']
        return np.max([d1, d2])
        
    sim = np.array(df.apply(similarity, axis= 1))
    best_match = np.argmin(sim)
    return {'best_match': df.iloc[best_match]['id']}
    
    
def twin(df, num):
    """Remove the top 'num' candidates from 'df'. For each removed candidate the most similar quasar among the remaining quasars is found. 
       Returns: an array of removed candidates with attached id of their twin. 
    """
    removed = df.iloc[:num]
    remaining = df.iloc[num:]
    
    best_match = removed.apply(lambda df_query: _twin(remaining, df_query), axis= 1, result_type= 'expand')
    removed = removed.join(best_match)
    return removed
    

removed= twin(data, 30)

Add the redshift information:

In [36]:
info = pd.read_csv('/pscratch/sd/j/jrobnik/quasars/quasar_info.txt', delim_whitespace= True)

ids = np.array(removed['best_match'], dtype= int)

info = info[info['id'].isin(ids)]
info = info.rename({'id': 'best_match'}, axis= 1)
removed2 = pd.merge(removed, info, on= 'best_match')
display(removed2)
removed2.to_csv('../data/twins.csv')

,logB,log_lik_ratio,white_periodogram,cycles,period,sigma,tau,A_const,A_sin,A_cos,success,data_points,T,log_prior_odds,id,best_match,redshift
0,28.008220,20.589591,79282.411750,4.785217,291.951536,0.404445,4.243316,19.122720,1.061551,0.150491,True,116,1397.051471,6.196449,366226,368194,1.136
1,26.411580,36.115200,701.148218,2.145789,644.612029,0.108759,0.893272,19.807833,-0.024083,0.115689,True,402,1383.201646,3.059500,263798,266602,3.020
2,23.896879,41.458955,762.703982,2.151729,672.461840,0.070448,1.227704,19.759194,0.103900,-0.073573,True,234,1446.955621,5.578081,316958,319778,0.968
3,22.692397,27.983853,499.342274,2.191961,660.140815,0.134961,1.522317,20.261716,-0.140253,0.009070,True,370,1447.002962,5.006433,304690,294347,2.781
4,22.233839,29.439039,478.611715,2.118093,683.170586,0.098069,1.522594,20.177484,-0.050697,-0.078000,True,445,1447.019004,5.491281,294564,305025,2.527
5,21.565749,27.858462,392.708048,2.703117,535.315123,0.110446,1.018172,20.109231,-0.093276,0.001276,True,413,1447.019292,4.898254,296911,298229,2.570
6,21.553779,28.131620,609.272665,2.433134,584.475158,0.132272,1.046000,20.108846,0.160755,0.094149,True,193,1422.106115,4.159160,263090,286888,2.495
7,21.124329,25.966691,370.800780,2.556315,541.075634,0.125920,0.996557,20.041977,0.056623,0.095468,True,381,1383.159901,4.083353,274155,264654,1.615
8,20.923072,13.316456,3858.339595,2.322948,502.257839,0.247381,6.321350,20.227376,-0.734339,-0.132289,True,54,1166.718871,2.208837,313614,296753,2.665
9,20.054897,32.953209,780.351474,2.751982,514.274213,0.029122,1.139361,17.238493,-0.025357,-0.003693,True,316,1415.273628,3.590840,234124,222255,2.589
